In [1]:
using CSV, DataFrames, Statistics, Clustering, Plots, Random, Gurobi, JuMP, DataStructures, StatsBase

In [2]:
data_prev = CSV.read("data/pred_Maria.csv",DataFrame)
data_opti = CSV.read("data/opti_Maria.csv",DataFrame)
data_logistic = CSV.read("data/opti_Maria.csv",DataFrame)
pred_logistic = CSV.read("data/pred_lr.csv",DataFrame)[:, "0"];

In [3]:
#number booked appointments
na = nrow(data_prev)
#number new appointments
nb = nrow(data_opti)
ntot = na + nb
#number doctors
nd = 15
#penalty added nurse
pen_nurse = 5
#previously_booked
s = data_prev.slot .+1 ;

In [4]:
a = Matrix(data_opti[:,41:45]);

In [42]:
p_oracle = ones(ntot,50)
for i = 1:na
    for j=1:50
        p_oracle[i,j] = 1 - data_prev.NoShow[i]
    end
end
for i = na+1:ntot
    for j=1:50
        p_oracle[i,j] = 1 - data_opti.NoShow[i-na]
    end
end

In [47]:
p_random = rand(ntot, 50);

p_logistic = ones(ntot,50)
for i = 1:na
    for j=1:50
        p_logistic[i,j] = 1 - pred_logistic[i]
    end
end
for i = na+1:ntot
    for j=1:50
        p_logistic[i,j] = 1 - pred_logistic[i-na]
    end
end

In [32]:
function optim_add_nurse(p, iteration)
    model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=180))
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, x[1:50], Bin)
    @variable(model, t[1:50])

    @objective(model, Min, sum(t)*pen_nurse)

    @constraint(model, [j=1:50], t[j] >= sum(y[i,j]*p[i,j] + x[j]*p[na + iteration + 1, j] for i=1:iteration + na) - nd) 
    @constraint(model, [j=1:50], t[j] >= 0)

    for j=1:50
        if j in a[iteration+1,:]
        else
            @constraint(model, x[j] == 0)
        end
    end

    @constraint(model, sum(x) == 1)

    optimize!(model)
    return objective_value(model), JuMP.value.(x)
end

optim_add_nurse (generic function with 1 method)

In [60]:
y = zeros(na, 50)
for i=1:na
    for j=1:50
        if s[i]==j
            y[i,j] = 1
        else
            y[i,j] = 0
        end
    end
end

In [61]:
for k=0:nb-1
    _, x = optim_add_nurse(p_logistic, k)
    y = [y;x']
end

Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only - expires 2022-08-18
Academic license - for non-commercial use only -

In [62]:
sum([max(sum(y[i,j]*p_oracle[i,j] for i=1:ntot) - nd, 0)*pen_nurse for j=1:50])

950.0